In [ ]:
# @title Load Modules

from collections import defaultdict
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import plotly.graph_objects as go

In [ ]:
# @title Model parameters

epsilon_val = 6.0
phi_mean_val = 0.5
beta_val = 1.5
g_val = 1
q0_val = 1

## Low-temperature Ansatz

In [ ]:
alpha, r, phi_max = symbols('\\alpha \\mathbf{r} \\phi_\\text{max}', real=True, positive=True)
V = symbols('V')

In [ ]:

phi_s_exp = phi_max * exp(-alpha * r**2)
phi_s_exp

In [ ]:

phi_s = symbols('\\phi_s')
phi_s


### Calcultate the integral of $\mathbf{r}^{2m} \phi_s^n$

In [ ]:
# @title #### Gaussian integral function: integral_r_2m_phi_s_n(m, n)

def integral_r_2m_phi_s_n (m, n):
  return phi_max**n * factorial(m) * pi / (alpha*n)**(m + 1)

In [ ]:
# @title #### Gaussian integral example m = 3, n = 2

integral_r_2m_phi_s_n(3,2)

#### Extract coeff, m, n from terms as $\textit{[coeff]} \cdot \mathbf{r}^{2m} \phi_s^n$

In [ ]:
def extract_coeff_m_n(expression):
  terms = expression.expand().as_ordered_terms()
  terms_dict = defaultdict(lambda: 0)
  for term in terms:
    coeff, _ = term.as_coeff_mul(phi_s,r)
    exponent_r = term.as_coeff_exponent(r)[1]
    exponent_phi_s = term.as_coeff_exponent(phi_s)[1]
    # exponent_r must be even
    assert exponent_r % 2 == 0

    terms_dict[(exponent_r/2, exponent_phi_s)] += coeff.simplify()


  terms_dict = sorted(terms_dict.items(), key=lambda x: (x[0][0], x[0][1]))
  terms_dict

  reconstructed_expression = 0
  for term in terms_dict:
    exponent_r, exponent_phi_s = term[0]
    coeff = term[1]
    reconstructed_expression += coeff * r**(2*exponent_r) * phi_s**exponent_phi_s

  try:
    assert N((expression - reconstructed_expression).simplify()) == 0
  except:
    print(terms_dict)
    raise Exception("Reconstructed expression does not match original expression")

  return terms_dict

In [ ]:
def calculate_expression_energy(expression):
  terms_dict = extract_coeff_m_n(expression)
  energy = 0
  for term in terms_dict:
    exponent_r, exponent_phi_s = term[0]
    coeff = term[1]
    energy += 1 / V * coeff * integral_r_2m_phi_s_n(exponent_r, exponent_phi_s)

  return energy

calculate_expression_energy(beta*phi_s)

#### Extra

In [ ]:
integrand_lin_terms = integrand_lin.expand().as_ordered_terms()
integrand_lin_terms

In [ ]:
integrand_lin_terms_dict = defaultdict(lambda: 0)

for term in integrand_lin_terms:
  coeff, _ = term.as_coeff_mul(phi_s,r)
  exponent_r = term.as_coeff_exponent(r)[1]
  exponent_phi_s = term.as_coeff_exponent(phi_s)[1]
  integrand_lin_terms_dict[(exponent_r, exponent_phi_s)] += coeff.simplify()
  # exponent_r must be even
  assert exponent_r % 2 == 0

integrand_lin_terms_dict = sorted(integrand_lin_terms_dict.items(), key=lambda x: (x[0][0], x[0][1]))
integrand_lin_terms_dict

In [ ]:
integrand_lin_comp = 0
for term in integrand_lin_terms_dict:
  exponent_r, exponent_phi_s = term[0]
  coeff = term[1]
  integrand_lin_comp += coeff * r**exponent_r * phi_s**exponent_phi_s

integrand_lin_comp

In [ ]:
free_energy_lin = 0
for term in integrand_lin_terms_dict:
  exponent_r, exponent_phi_s = term[0]
  coeff = term[1]
  free_energy_lin += 1 / v * coeff * integral_r_2m_phi_s_n(exponent_r, exponent_phi_s)

#free_energy_lin = free_energy_lin.simplify()
free_energy_lin

In [ ]:
integrand_terms = integrand.expand().as_ordered_terms()
integrand_terms

In [ ]:
integrand_terms_dict = defaultdict(lambda: 0)

for term in integrand_terms:
  coeff, _ = term.as_coeff_mul(phi_s,r)
  exponent_r = term.as_coeff_exponent(r)[1]
  exponent_phi_s = term.as_coeff_exponent(phi_s)[1]
  integrand_terms_dict[(exponent_r, exponent_phi_s)] += coeff.simplify()
  # exponent_r must be even
  assert exponent_r % 2 == 0

integrand_terms_dict = sorted(integrand_terms_dict.items(), key=lambda x: (x[0][0], x[0][1]))
integrand_terms_dict

In [ ]:
integrand_comp = 0
for term in integrand_terms_dict:
  exponent_r, exponent_phi_s = term[0]
  coeff = term[1]
  integrand_comp += coeff * r**exponent_r * phi_s**exponent_phi_s

integrand_comp

In [ ]:
(integrand - integrand_comp).simplify()

### Integrals of gradient terms

#### $∇^2 \phi_s$

In [ ]:
# @title ##### $\nabla^2\phi_s(\mathbf{r}, \alpha, \phi_\text{max})$

diff_r = diff(phi_s_exp, r)
diff_r2 = diff(diff_r, r)
lap_phi_s = diff_r2 + diff_r/r
lap_phi_s

In [ ]:
# @title ##### $\nabla^2\phi_s(\mathbf{r}, \alpha, \phi_s)$

lap_phi_s = ((lap_phi_s/phi_s_exp).simplify()*phi_s).expand()
lap_phi_s

In [ ]:
# @title ##### $\frac{2 \beta}{2 V} \int d \mathbf{r} \; \phi_s\nabla^2\phi_s$ (analytic vs numeric)

N(calculate_expression_energy(beta * phi_s * lap_phi_s).subs({alpha: alpha_atom, phi_max: amp_atom, beta: beta_val, V: V_val})), sim_energy.energy_lin_1.mean()

#### $∇^4 \phi_s$

In [ ]:
# @title ##### $\nabla^4\phi_s(\mathbf{r}, \alpha, \phi_\text{max})$

diff_r_lap_phi_s = diff(lap_phi_s.subs({phi_s: phi_s_exp}), r)
diff_r2_lap_phi_s = diff(diff_r_lap_phi_s, r)
lap2_phi_s = (diff_r2_lap_phi_s + diff_r_lap_phi_s/r).simplify()
lap2_phi_s

In [ ]:
# @title ##### $\nabla^4\phi_s(\mathbf{r}, \alpha, \phi_s)$

lap2_phi_s = ((lap2_phi_s/phi_s_exp).simplify()*phi_s).expand()
lap2_phi_s

In [ ]:
# @title ##### $\frac{\beta}{2 V} \int d \mathbf{r} \; \phi_s\nabla^4\phi_s$ (analytic vs numeric)

N(calculate_expression_energy(beta * phi_s * lap2_phi_s/2).subs({alpha: alpha_atom, phi_max: amp_atom, beta: beta_val, V: V_val})), sim_energy.energy_lin_2.mean()

###  Integral of $\phi_s \ln \phi_s$ (large $\alpha$, no overlap)

In [ ]:
phi_s_log_phi_s = (phi_s_exp*(log(phi_max) + log(phi_s_exp/phi_max))).simplify().expand()
phi_s_log_phi_s

phi_s_log_phi_s = ((phi_s_log_phi_s/phi_s_exp).simplify())*phi_s_exp
phi_s_log_phi_s

phi_s_log_phi_s = (phi_s_log_phi_s/phi_s_exp).simplify()*phi_s
phi_s_log_phi_s

In [ ]:
# @title ##### $\frac{1}{V} \int d \mathbf{r} \; \phi_s \ln \phi_s$ (analytic vs numeric)

N(calculate_expression_energy(phi_s_log_phi_s).subs({alpha: alpha_atom, phi_max: amp_atom, beta: beta_val, V: V_val})), sim_energy.energy_ln.mean()

### Total free energy per unit volume

$$
F(\phi_s) = \frac{1}{V} \int_\text{unit cell} d\vec{r} \left[ \phi_s \ln \phi_s + \frac{1}{2} \beta \phi_s (\nabla^4 + 2 {q_0}^2 \nabla^2 + {q_0}^4) \phi_s + \frac{1}{2} (-\epsilon) {\phi_s}^2 + \frac{1}{3} (-g) {\phi_s}^3 + \frac{1}{4} {\phi_s}^4 \right]
$$

In [ ]:
nabla, beta, q0, epsilon, g = symbols('\\nabla \\beta q_0 \\epsilon g', real=True, positive=True)

In [ ]:
phi_s * log(phi_s) + Rational(1,2) * beta * phi_s * (nabla**4 * phi_s + 2 * q0**2 * nabla**2 * phi_s  + q0**4 * phi_s) + Rational(1,2) * (-epsilon) * phi_s**2 + Rational(1,3) * (-g) * phi_s**3 + Rational(1,4) * phi_s**4

In [ ]:
integrand_phi_log_phi = phi_s_log_phi_s
integrand_phi_log_phi

In [ ]:
integrand_lin = Rational(1,2) * beta * phi_s * (lap2_phi_s + 2 * q0**2 * lap_phi_s)
integrand_lin

In [ ]:
integrand_poly = Rational(1,2) * beta * q0**4 * phi_s**2 + Rational(1,2) * (-epsilon) * phi_s**2 + Rational(1,3) * (-g) * phi_s**3 + Rational(1,4) * phi_s**4
integrand_poly

In [ ]:
integrand = integrand_phi_log_phi + integrand_lin + integrand_poly
integrand = integrand.expand().collect(phi_s)
integrand

In [ ]:
# @title ##### $\frac{1}{V} \int d \mathbf{r} \; \phi_s \ln \phi_s$ (analytic vs numeric)

N(calculate_expression_energy(integrand).subs({alpha: alpha_atom, phi_max: amp_atom, beta: beta_val, epsilon: epsilon_val, g: g_val, q0: q0_val, V: V_val})), sim_energy.f.mean()

## Calculate the free energy density

In [ ]:
free_energy = calculate_expression_energy(integrand)
free_energy

## Minimize free energy

$$
\begin{align}
\overline\phi &= \frac{1}{V} \int d \mathbf{r} \phi_s(\mathbf{r}) \\[1.25em]
\overline\phi &= \frac{1}{V} \int d \mathbf{r} \phi_\text{max} e^{-\alpha r^2} \\[1.25em]
\overline\phi &= \frac{\pi}{\alpha V} \phi_\text{max} \\[1.25em]
\Rightarrow \phi_\text{max} &= \frac{\alpha V}{\pi} \overline\phi
\end{align}
$$

In [ ]:
phi_mean, q2 = symbols('\\overline{\\phi}_s q_s^2', real=True)   # , positive=True)

In [ ]:
free_energy_phi_mean_q = free_energy.subs({phi_max: alpha*V*phi_mean/pi}).subs({V: 8 * pi**2 / sqrt(3) / q2}).expand()
free_energy_phi_mean_q

In [ ]:
q_eq = together(diff(free_energy_phi_mean_q, q2))
_, q_eq_denom = fraction(q_eq)
q_eq = q_eq * q_eq_denom # expand_log(q_eq * q_eq_denom)
q_eq_poly = Poly(q_eq.expand(), q2, alpha)
q_eq_lead_coeff = q_eq_poly.LC()
q_eq = (q_eq / q_eq_lead_coeff).expand()
q_eq_q_exponent_min = np.array(q_eq_poly.monoms())[:,0].min()
q_eq = q_eq / q2**q_eq_q_exponent_min
q_eq_alpha_exponent_min = np.array(q_eq_poly.monoms())[:,1].min()
q_eq = (q_eq / alpha**q_eq_alpha_exponent_min).simplify()
q_eq

In [ ]:
alpha_eq = together(diff(free_energy_phi_mean_q, alpha))
_, alpha_eq_denom = fraction(alpha_eq)
alpha_eq = alpha_eq * alpha_eq_denom # expand_log(alpha_eq * alpha_eq_denom)
alpha_eq_poly = Poly(alpha_eq.expand(), alpha, q2)
alpha_eq_lead_coeff = alpha_eq_poly.LC()
alpha_eq = (alpha_eq / alpha_eq_lead_coeff).expand()
alpha_eq_alpha_exponent_min = np.array(alpha_eq_poly.monoms())[:,0].min()
alpha_eq = alpha_eq / alpha**alpha_eq_alpha_exponent_min
alpha_eq_q_exponent_min = np.array(alpha_eq_poly.monoms())[:,1].min()
alpha_eq = alpha_eq / q2**alpha_eq_q_exponent_min
alpha_eq

In [ ]:
q_eq_poly = Poly(q_eq, q2)  #Poly(q_eq, q2, alpha)
q_eq_poly

In [ ]:
alpha_eq_poly = Poly(alpha_eq, alpha)
alpha_eq_poly

In [ ]:
N(alpha_eq_poly.subs({q2: 1, phi_mean: sim_energy.phi.mean(), beta: beta_val, q0: q0_val, epsilon: epsilon_val, g: g_val}))